## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Read-in

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

## Clean Data

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [6]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [10]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon
1,Portofino Towers\r300 S Pointe Dr 3105\rMiami ...,4/11/2023,482,"$8,800,000.00",5450,"$1,614.68",John Lennon,South Pointe Drive Realty Inc,J. Jacob Abdala,Legacy Plus Realty Inc,Portofino Towers 300 S Pointe Dr 3105 Miami Beach,Portofino Towers,300 S Pointe Dr 3105 Miami Beach,"(25.7677055, -80.1348768)",25.767706,-80.134877
2,Brickell Flatiron Condo\r1000 Brickell Plz TH1...,4/10/2023,65,"$4,500,000.00",5251,"$1,126.41",Nelson Gonzalez PA,BHHS EWM Realty,Jordan Lederman,Fortune International Realty,Brickell Flatiron Condo 1000 Brickell Plz TH18...,Brickell Flatiron Condo,1000 Brickell Plz TH1802 Miami,"(25.7642427, -80.1927022)",25.764243,-80.192702
3,1 Hotel & Homes\r102 24th St 1244\rMiami Beach,4/14/2023,240,"$4,500,000.00",2075,"$2,476.61",Jennifer Restrepo,EXP Realty LLC,Patty Marun,Suma Luxury Realty,1 Hotel & Homes 102 24th St 1244 Miami Beach,1 Hotel & Homes,24th St 1244,"(25.799494, -80.1268322)",25.799494,-80.126832
4,Mei Condo\r5875 Collins Ave 2102\rMiami Beach,4/11/2023,8,"$3,850,000.00",1725,"$2,231.88",Ryan Mendell,Maxwell E Realty Inc,Ryan Mendell,Maxwell E Realty Inc,Mei Condo 5875 Collins Ave 2102 Miami Beach,Mei Condo,5875 Collins Ave 2102 Miami Beach,"(25.8408333, -80.1205556)",25.840833,-80.120556
5,Bellini Condo\r10225 Collins Ave 902\rBal Harbour,4/12/2023,263,"$3,450,000.00",3820,$903.14,David Siddons,Douglas Elliman,Lydia Eskenazi PA,One Sotheby's Int'l Realty,Bellini Condo 10225 Collins Ave 902 Bal Harbour,Bellini Condo,10225 Collins Ave 902 Bal Harbour,"(25.8964898, -80.12326499999999)",25.896490,-80.123265
6,Jade Ocean\r17121 Collins Ave 1405\rSunny Isle...,4/11/2023,21,"$2,700,000.00",1933,"$1,396.79",Cintia Thormann,"Compass Florida, LLC",Sergio Chanteiro,First Service Realty ERA,Jade Ocean 17121 Collins Ave 1405 Sunny Isles ...,Jade Ocean,17121 Collins Ave 1405 Sunny Isles Beach,"(25.9344014, -80.1210054)",25.934401,-80.121005
7,Key Colony\r151 Crandon Blvd 824\rKey Biscayne,4/13/2023,195,"$2,360,000.00",3064,$770.24,Geysa Guarconi,Luft Realty,Jorge Boj,Great Properties Int'l LLC,Key Colony 151 Crandon Blvd 824 Key Biscayne,Key Colony,151 Crandon Blvd 824 Key Biscayne,"(25.6972293, -80.1594442)",25.697229,-80.159444
8,The Courts at South Beach\r100 Jefferson Ave 1...,4/14/2023,78,"$1,865,000.00",2160,$863.43,Jorn Lahmann,Charles Rutenberg Realty FTL,David Nguah,BHHS EWM Realty,The Courts at South Beach 100 Jefferson Ave 10...,The Courts at South Beach,100 Jefferson Ave 10014 Miami Beach,"(25.7705701, -80.13698529999999)",25.770570,-80.136985
9,Icon Brickell No 1\r465 Brickell Ave 3801\rMiami,4/13/2023,91,"$1,700,000.00",1825,$931.51,Kara Walter,Royal Palm Realty Of S Florida,Carlo Ortiz,Xcellence Realty,Icon Brickell No 1 465 Brickell Ave 3801 Miami,Icon Brickell No,1 465 Brickell Ave 3801 Miami,"(25.7689974, -80.18854999999999)",25.768997,-80.188550
10,0Trump Towers\r16001 Collins Ave 2702\rSunny I...,4/14/2023,69,"$1,650,000.00",1979,$833.75,Marco Cunha,Prestige Luxury Real Estate,Lucila Maltagliati,One Sotheby's Int'l Realty,0Trump Towers 16001 Collins Ave 2702 Sunny Isl...,Trump Towers,16001 Collins Ave,"(25.9237465, -80.1216974)",25.923747,-80.121697


In [9]:
df.at[3,'building_name']=('1 Hotel & Homes')
df.at[3,'address_only']=('24th St 1244')

df.at[10,'building_name']=('Trump Towers')
df.at[10,'address_only']=('16001 Collins Ave')

## Format Data

In [11]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

In [12]:
df['int_Sale_Price'] = df['Sale Price'].str.replace('$','',regex=True)
df['int_Sale_Price'] = df['int_Sale_Price'].str.replace(',','',regex=True)
df['int_Sale_Price'] = pd.to_numeric(df['int_Sale_Price'])

df['price_per_sqft'] = df['price_per_sqft'].str.replace('$','',regex=True)
df['price_per_sqft'] = df['price_per_sqft'].str.replace(',','',regex=True)
df['price_per_sqft'] = pd.to_numeric(df['price_per_sqft'])

df['days_on_market'] = pd.to_numeric(df['days_on_market'])

## Color-code top sale

In [13]:
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [14]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

In [15]:
def popup_html(row):
    building_name = row['building_name']
    price = row['Sale Price']
    days_on_market = row['days_on_market']
    listing_agent = row['Agent']
    buyers_agent = row['Buyer Agent']
    psf = row['price_per_sqft']
    address = row['address_only']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>{}'''.format(price) + '''<br>
    <strong>Days on Market: </strong>{}'''.format(days_on_market) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(listing_agent) + '''<br>
    <strong>Buyer's Agent: </strong>{}'''.format(buyers_agent) + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(psf) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [16]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('April 9th - April 15th')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [17]:
m.save('index.html')

## Data snagger

In [18]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [19]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

Most Expensive
Portofino Towers , 300 S Pointe Dr 3105 Miami Beach | Price $8,800,000 | $1,615 psf | Listing agent: John Lennon with South Pointe Drive Realty Inc | Buyer's agent: J. Jacob Abdala with Legacy Plus Realty Inc | Days on market: 482
Least Expensive
Trump Towers, 16001 Collins Ave | Price $1,650,000 | $834 psf | Listing agent: Marco Cunha with Prestige Luxury Real Estate | Buyer's agent: Lucila Maltagliati with One Sotheby's Int'l Realty | Days on market: 69
Highest Price Per Square Foot
1 Hotel & Homes, 24th St 1244 | Price $4,500,000 | $2,477 psf | Listing agent: Jennifer Restrepo with EXP Realty LLC | Buyer's agent: Patty Marun with Suma Luxury Realty | Days on market: 240
Lowest Price Per Square Foot
Key Colony , 151 Crandon Blvd 824 Key Biscayne | Price $2,360,000 | $770 psf | Listing agent: Geysa Guarconi with Luft Realty | Buyer's agent: Jorge Boj with Great Properties Int'l LLC | Days on market: 195
Most Days on Market
Portofino Towers , 300 S Pointe Dr 3105 Miami B

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [20]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [21]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_04172023
